In [ ]:
import pandas as pd
from transformers import pipeline

df = pd.read_csv('total_review_sentiments.csv')

df['model'] = df['model'].str.lower()

def summarize_phone_sentiments(model_name):
    phone_reviews = df[df['model'] == model_name]

    if phone_reviews.empty:
        raise ValueError(f"No reviews found for the model '{model_name}'.")

    aspect_columns = [col for col in phone_reviews.columns if col.endswith("_full_phrase")]
    if not aspect_columns:
        raise ValueError(f"No aspect columns found in the dataset for model '{model_name}'.")

    full_phrases = []
    for col in aspect_columns:
        full_phrases.extend(phone_reviews[col].dropna().tolist())
    combined_text = " ".join(full_phrases)

    if len(combined_text.split()) < 10:
        return "Not enough data to generate a meaningful summary."

    max_input_length = 1024
    combined_text = combined_text[:max_input_length]

    summarizer = pipeline('summarization', model='facebook/bart-large-cnn')

    max_length = min(300, len(combined_text.split()) + 20)

    summary = summarizer(combined_text, max_length=max_length, min_length=50, do_sample=False)
    return summary[0]['summary_text']

if __name__ == "__main__":
    unique_models = df['model'].unique()
    print("Available phone models:")
    for idx, model in enumerate(unique_models, start=1):
        print(f"{idx}. {model}")

    model_name = input("\nEnter the phone model name: ").strip().lower()

    try:
        summary = summarize_phone_sentiments(model_name)
        print(f"\nSummary for '{model_name}':\n{summary}")
    except ValueError as e:
        print(f"Error: {e}")
